In [85]:
import numpy as np
import random

# Data preparation

In [94]:
vocab = 'abcdefghigklmnopqrstuvwxyz '
len(vocab)

26

In [95]:
def find(letter, vocab):
    try:
        temp = vocab.index(letter)
    except ValueError:
        temp = -1
    return temp

def WordToAlphabet(word):
    vec = np.zeros((len(word), len(vocab), 1))
    for i, letter in enumerate(word):
        vec[i, find(letter, vocab)] = 1.0
    return vec

def vecToWord(vec):
    word = [None for _ in range(vec.shape[0])]
    for i in range(vec.shape[0]):
        temp = np.argmax(vec[i])
        word[i] = vocab[temp]    
    return word

In [153]:
x = []
y = []
with open('dataset/texts/simple_words.txt', 'r') as file:
    read_data = file.readline()
    read_data = read_data.lower()
    print(read_data[:])
    #lines = file.readlines()
    while(read_data):
        x.append(WordToAlphabet(read_data[:-1]))
        y.append(WordToAlphabet(read_data[1:]))
        read_data = file.readline()
        read_data = read_data.lower()
        
data = list(zip(x, y))
random.shuffle(data)

ability



In [255]:
"""class ActivationSoftmax():
    @staticmethod
    def f(z):
        a = np.exp(z)
        return a / np.sum(a)
    @staticmethod
    def prime(z):
"""
def softmax(z):
    a = np.exp(z)
    return a / np.sum(a)

class CrossEntropyCost():
    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)))
    def delta(z, a, y):
        return a - y
    
class RNNetwork():
    def __init__(self, vocab_size, hidden_size, bptt_trunc = 4, cost = CrossEntropyCost):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.bptt_trunc = bptt_trunc
        self.Wxh = np.random.randn(hidden_size, vocab_size)
        self.Whh = np.random.randn(hidden_size, hidden_size)
        self.Wha = np.random.randn(vocab_size, hidden_size)
        self.h = np.zeros((hidden_size, 1))
        #self.bh = np.zeros((hidden_size, 1))
        #self.ba = np.zeros((vocab_size, 1))
        self.cost = cost
        # U Wxh
        # W Whh
        # V Wha
        
    def feedforward(self, x):
        time_steps = len(x)
        z = np.zeros((time_steps, self.vocab_size, 1))
        #h = np.zeros((time_steps + 1, self.hidden_size, 1))
        #h[-1] = np.zeros(h[0].shape)
        for t in range(time_steps):
            self.h = np.tanh(self.Wxh @ x[t] + self.Whh @ self.h)# + self.bh)
            """z[t] = self.Wha @ self.h #+ self.ba
        return softmax(z)"""
    #why not softmax(z[t])
            z[t] = softmax(self.Wha @ self.h)
        return z
    
    def feedforwardWithData(self, x):
        time_steps = len(x)
        z = np.zeros((time_steps, self.vocab_size, 1))
        h = np.zeros((time_steps + 1, self.hidden_size, 1))
        h[-1] = np.zeros(h[0].shape)
        for t in range(time_steps):
            self.h = np.tanh(self.Wxh @ x[t] + self.Whh @ self.h)# + self.bh)
            z[t] = softmax(self.Wha @ self.h) #+ self.ba
            h[t] = self.h
        return z, h
    
    def loss(self, x, y):
        return self.cost.fn(self.feedforward(x), y) / len(x)
    
    def total_loss(self, data):
        n = len(data)
        cost = 0.0
        for (x, y) in data:
            cost += self.loss(x, y)
        return cost / n
    
    def BPTT(self, data, learning_rate = 1):
        params = [self.Wxh, self.Whh, self.Wha]
        for (x, y) in data:
            nabla_params = self.backprop(x, y)
            for (param, nabla_param) in zip(params, nabla_params):
                param -= learning_rate * nabla_param
                
    def predict(self, initial, time_steps = 10):
        vectorized = WordToAlphabet(initial)
        for t in range(len(vectorized)):
            self.h = np.tanh(self.Wxh @ vectorized[t] + self.Whh @ self.h)# + self.bh)
        for t in range(time_steps):
            self.h = np.tanh(self.Wxh @ x[t] + self.Whh @ self.h)# + self.bh)
            z[t] = self.Wha @ self.h #+ self.ba
            h[t] = self.h
        return 0
    
    def backprop(self, x, y):
        time_steps = len(x)
        nabla_Wxh = np.zeros(self.Wxh.shape)
        nabla_Whh = np.zeros(self.Whh.shape)
        nabla_Wha = np.zeros(self.Wha.shape)
        y_hat, h = self.feedforwardWithData(x)
        
        delta = np.dot(self.Wha.T, y_hat[time_steps - 1] - y[time_steps - 1]) #delta = dl/dh
        
        for t in range(time_steps):
            nabla_Wha += np.dot(y_hat[t] - y[t], h[t].T)
        for t in range(time_steps - 2, -1, -1): #already have done the last time step
            delta_help = np.dot(self.Wha.T, y_hat[t] - y[t])
            helper = np.diag((np.ones(h[t].shape) - h[t] ** 2)[:, 0])
            #print(helper.shape, delta.shape)
            delta = np.dot(self.Whh.T, helper) @ delta + delta_help
            nabla_Whh += np.dot(helper @ delta, h[t - 1].T)
            nabla_Wxh += np.dot(helper @ delta, x[t - 1].T)
        return [nabla_Wxh, nabla_Whh, nabla_Wha]

Проверить правильность формул<br>
Построить график loss(% complete)<br>
Сменить базу данных<br>
Добавить biases<br>

In [328]:
net = RNNetwork(len(vocab), 15)

In [335]:
net.BPTT(data, 0.005)
net.total_loss(data)

3.1244035740168687

In [336]:
i = 14
out, h = net.feedforwardWithData(np.array(data[i][0]))
vecToWord(out), vecToWord(data[i][1])

(['e', 'i', 'z', 'z', 'z', 'z', 'z', 'l', 'z', 'n', 'h'],
 ['e', 'r', 'f', 'o', 'r', 'm', 'a', 'n', 'c', 'e', 'z'])

In [338]:
net.total_loss(data)

3.1186227957327524